In [1]:
import numpy as np
import pandas as pd
import math
import string
from time import time


directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1),  (1, 0), (1, 1)]

class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_word = False

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_word = True

    def search(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                return False
            node = node.children[char]
        return node.is_word

    def starts_with(self, prefix):
        node = self.root
        for char in prefix:
            if char not in node.children:
                return False
            node = node.children[char]
        return True

trie = Trie()
with open('dictionary_actual_game.txt', 'r') as file:
    for word in file.read().splitlines():
        trie.insert(word)
        
keys = list(string.ascii_lowercase)
score = [1,3,3,2,1,4,2,4,3,8,5,5,3,2,1,3,10,1,1,1,2,4,4,8,4,10]
letter_to_int = dict(zip(keys, range(1, 27)))
letter_to_int["N"] = 0

dictionary_df = pd.read_csv("dictionary_df2.csv")
dictionary_df.loc[160413, "word"] = "null"
dictionary_df.loc[154353, "word"] = "nan"
dictionary_df = dictionary_df[dictionary_df.columns[1:]]
score_dict = dict(zip(dictionary_df["word"], dictionary_df["score"]))

def tiles_remove(tiles, board):
    board[tiles[:, 0], tiles[:, 1]] = "N"
    return board

def gravity(board_1):
    # Gravity down
    row_index, col_index = np.where(board_1 == "N")
    indices = list(zip(row_index, col_index))
    for r, c in indices:
        column = board_1[:r + 1, c].copy()
        if r > 0:
            column[1:] = column[:-1]
            column[0] = "N"
        board_1[:r + 1, c] = column
    # Gravity right
    row_index, col_index = np.where(board_1 == "N")
    indices = list(zip(row_index, col_index))
    for r, c in indices:
        row = board_1[r, :c + 1].copy()
        if c > 0:
            row[1:] = row[:-1]
            row[0] = "N"
        board_1[r, :c + 1] = row
    return board_1

class WordSoup_state:
    def __init__(self, board, S=0, word_lst=[], depth=0):
        self.board = board
        self.score = S
        self.word_lst = word_lst
        self.depth = depth
        self.board_array = np.array(board)

    def get_legal_actions(self):
        rows, cols = len(self.board), len(self.board[0])
        result = []

        def dfs(i, j, path, indices, visited, node):
            if node.is_word:
                result.append((path, indices))  # Store the word and its indices
            for dx, dy in directions:
                x, y = i + dx, j + dy
                if 0 <= x < rows and 0 <= y < cols and (x, y) not in visited:
                    char = self.board[x][y]
                    if char in node.children:
                        visited.add((x, y))
                        dfs(x, y, path + char, indices + [(x, y)], visited, node.children[char])
                        visited.remove((x, y))

        for i in range(rows):
            for j in range(cols):
                char = self.board[i][j]
                if char in trie.root.children:
                    dfs(i, j, char, [(i, j)], {(i, j)}, trie.root.children[char])

        return result

    def is_game_over(self):
        found_words = self.get_legal_actions()
        if len(found_words) > 0:
            return False
        return True

    def game_result(self, Score):
        if self.score >= Score:
            return 1
        return -1

    def move(self, action):
        word, indices = action
        _ = tiles_remove(np.array(indices), self.board_array.copy())
        new_board = gravity(_)
        new_score = self.score + score_dict[word]
        if np.sum(new_board != "N") == 0:
            new_score += 500
        new_board = new_board.tolist()
        new_word_lst = self.word_lst.copy()
        new_word_lst.append(word)

        return WordSoup_state(new_board, new_score, new_word_lst, self.depth+1)

In [2]:
categories = list(range(1, 27))
probabilities = [
    76, 28, 30, 37, 112, 16, 26, 27, 51, 9, 24, 40, 32, 62, 88, 37, 11, 60, 66, 64, 39, 16, 18, 12, 22, 16
]
probabilities = np.array(probabilities)
probabilities = probabilities/sum(probabilities)
dictionary = {
    0: "N",
    1: "a", 2: "b", 3: "c", 4: "d", 5: "e", 6: "f", 7: "g", 8: "h", 9: "i", 10: "j", 11: "k", 12: "l", 13: "m",
    14: "n", 15: "o", 16: "p", 17: "q", 18: "r", 19: "s", 20: "t", 21: "u", 22: "v", 23: "w", 24: "x", 25: "y",
    26: "z"
}

def board_generator(n=9, seed=0, repeat=True):
    if repeat:
        np.random.seed(seed)
        
    bottom = np.random.randint(math.ceil(np.sqrt(n)), 9)
    row = np.random.choice(categories, bottom, p=probabilities)
    row_ = np.zeros(9).astype("int32")
    row_[-len(row):] = row
    row_ = row_.tolist()
    vector_lst = [row_]
    l = bottom
    b = l
    while l < n:
        num = (-1)*np.random.randint(1, min(n-l+1, b+1))
        b = (-1)*num
        row = np.random.choice(categories, b, p=probabilities)
        row_ = np.zeros(9).astype("int32")
        row_[-len(row):] = row
        row_ = row_.tolist()
        vector_lst = [row_] + vector_lst
        l-=num
        
    board = np.vectorize(dictionary.get)(vector_lst)
    return board.tolist()

In [4]:
def ExhaustiveSearch(state, step=1):
    L = 0
    for l in state.board:
        for i in l:
            L+=1
    untried_actions = state.get_legal_actions()
    if not untried_actions:
        return
    untried_actions_sorted = sorted(untried_actions, key=lambda x: score_dict[x[0]], reverse=True)
    best_score_so_far = score_dict[untried_actions_sorted[0][0]]
    best_ = []
    for action in untried_actions_sorted:
        if score_dict[action[0]]==best_score_so_far:
            best_.append(action)
        else:
            break
    result = [state.move(action) for action in best_]
    if step == 1:
        return result
    else:
        queue = [state.move(action) for action in untried_actions_sorted]
        best = []
    
    while queue:
        state = queue.pop(0)
        y = best_score_so_far - state.score
        untried_actions = state.get_legal_actions()
        if len(untried_actions) == 0:
            continue
        length = 0
        for l in state.board:
            for i in l:
                if i != "N":
                    break
                length+=1
        
        def Score_Cal(x):
            if len(x[0])==(L-length):
                return score_dict[x[0]]+500
            else:
                return score_dict[x[0]]
        untried_actions_sorted = sorted(untried_actions, key=lambda x: Score_Cal(x), reverse=True)
        s_ = Score_Cal(untried_actions_sorted[0])
        if s_ > y:
            best_score_so_far = state.score + s_
            best = []
            for x in untried_actions_sorted:
                if Score_Cal(x)==s_:
                    best.append((state, x))
                else:
                    break
        if state.depth < step - 1:
            _ = [state.move(action) for action in untried_actions_sorted]
            if _:
                queue = _ + queue
            else:
                continue
    if best:
        result = [state.move(action) for state, action in best]
    return result

In [77]:
%%time
np.random.seed(0)
loops = 500
time_lst = []
length_lst = []

for l in range(loops):
    length = np.random.randint(9, 16)
    length_lst.append(length)
    board = board_generator(n=length, repeat=False)
    initial_state = WordSoup_state(board)
    p1 = time()
    if length == 9:
        ExhaustiveSearch(initial_state, 3)
        p2 = time()
        time_lst.append(p2-p1)
        continue
    elif length <= 12:
        ExhaustiveSearch(initial_state, 4)
        p2 = time()
        time_lst.append(p2-p1)
        continue
    elif length <= 15:
        ExhaustiveSearch(initial_state, 5)
        p2 = time()
        time_lst.append(p2-p1)

CPU times: user 22min 28s, sys: 29.9 s, total: 22min 58s
Wall time: 22min 37s


In [78]:
time_lst

[0.03961515426635742,
 0.0339970588684082,
 69.33325791358948,
 5.2506020069122314,
 0.011234045028686523,
 0.006461143493652344,
 0.0026769638061523438,
 0.0019469261169433594,
 0.05878806114196777,
 8.826085090637207,
 0.017412185668945312,
 0.10005784034729004,
 0.3325769901275635,
 0.008228063583374023,
 5.77719521522522,
 1.0367558002471924,
 0.020985126495361328,
 0.40224790573120117,
 0.027048110961914062,
 0.043142080307006836,
 0.2183680534362793,
 0.00011181831359863281,
 0.006072044372558594,
 0.0006711483001708984,
 0.16728997230529785,
 1.123180866241455,
 0.023300886154174805,
 1.3144431114196777,
 0.00043010711669921875,
 0.01922893524169922,
 0.0036859512329101562,
 0.008608579635620117,
 1.0680770874023438,
 0.00711512565612793,
 0.004408121109008789,
 2.9754631519317627,
 0.7268500328063965,
 0.4933769702911377,
 0.16479706764221191,
 0.43053293228149414,
 0.013516664505004883,
 8.834290742874146,
 0.05288505554199219,
 0.004205942153930664,
 20.46245503425598,
 0.335

In [79]:
df = pd.DataFrame([length_lst, time_lst]).T
df.columns = ["length", "time"]
[(i, np.mean(d["time"])) for i, d in df.groupby("length")]

[(9.0, 0.015600798786550329),
 (10.0, 0.06935788336254302),
 (11.0, 0.16796722412109374),
 (12.0, 0.6692200479373126),
 (13.0, 3.7052270812647685),
 (14.0, 2.8590154859084116),
 (15.0, 11.298970947140141)]

In [80]:
df

,length,time
0,13.0,0.039615
1,13.0,0.033997
2,15.0,69.333258
3,15.0,5.250602
4,9.0,0.011234
...,...,...
495,14.0,6.224421
496,11.0,0.181303
497,12.0,0.008703
498,9.0,0.009361
